In [2]:
!pip3 install -r /content/drive/MyDrive/workspace/cur-sol/train/requirements.txt --quiet

import IPython
print( 'restarting kernel...' )
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 53.7 MB/s eta 0:00:00
restarting kernel...


{'status': 'ok', 'restart': True}

In [10]:
from google.cloud.bigquery import magics
import google.auth
from numpy.core.arrayprint import DatetimeFormat
credentials, project = google.auth.default()

In [2]:
PROJECT_ID = "kallogjeri-project-345114" # @param {type:"string"} <---CHANGE THESE
PROJECT_NUMBER = '273872083706' # @param {type:"string"} <---CHANGE THESE
SERVICE_ACCOUNT_NAME ='273872083706-compute@developer.gserviceaccount.com' # @param {type:"string"} <---CHANGE THESE
MODEL_NAME ='deepchem_predictions' # @param {type:"string"} <---CHANGE THESE
LOCATION ='us-central1' # @param {type:"string"} <---CHANGE THESE

%env PROJECT_ID=$PROJECT_ID
%env PROJECT_NUMBER=$PROJECT_NUMBER
%env SERVICE_ACCOUNT_NAME=$SERVICE_ACCOUNT_NAME
%env MODEL_NAME=$MODEL_NAME
%env LOCATION=$LOCATION


IMAGE_URI=f"gcr.io/{PROJECT_ID}/deepchem:latest"
%env IMAGE_URI=$IMAGE_URI

LOCATION = "us-central1"  # @param {type:"string"}

DRIVE_HOME="/content/drive" # @param {type:"string"} <---CHANGE THESE
WORKSPACE_HOME = f"{DRIVE_HOME}/MyDrive/workspace"
%env WORKSPACE_HOME=$WORKSPACE_HOME


import sys
IN_COLAB = 'google.colab' in sys.modules
%env IN_COLAB=$IN_COLAB
!gcloud config set project $PROJECT_ID -q

!echo {WORKSPACE_HOME}

env: PROJECT_ID=kallogjeri-project-345114
env: PROJECT_NUMBER=273872083706
env: SERVICE_ACCOUNT_NAME=273872083706-compute@developer.gserviceaccount.com
env: MODEL_NAME=deepchem_predictions
env: LOCATION=us-central1
env: IMAGE_URI=gcr.io/kallogjeri-project-345114/deepchem:latest
env: WORKSPACE_HOME=/content/drive/MyDrive/workspace
env: IN_COLAB=True
Updated property [core/project].
/content/drive/MyDrive/workspace


In [3]:
from google.colab import drive
drive.mount(f'{DRIVE_HOME}', force_remount=True)

Mounted at /content/drive


docker pull deepchemio/deepchem:latest
docker tag deepchem gcr.io/[GCP_PROJECT]/deepchem
docker push gcr.io/[GCP_PROJECT]/deepchem



In [4]:
from deepchem.models.keras_model import KerasModel
import deepchem as dc
import tensorflow as tf
print(f'dc: {dc.__version__}')
print(f'tf: {tf.__version__}')


dc: 2.7.1
tf: 2.12.0


In [5]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [6]:
from deepchem.models.graph_models import GraphConvModel
model_dir = f'{WORKSPACE_HOME}/cur-solubility/models/'
model = GraphConvModel(model_dir=model_dir, n_tasks=1, mode='regression', dropout=0.2)
model.fit(train_dataset, nb_epoch=15, deterministic=True)
model.save_checkpoint()


In [7]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.10260247942367831}
test set score: {'pearson_r2_score': 0.030558816164417985}


In [8]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)
latest = tf.train.latest_checkpoint(model_dir)
latest

2.12.0


'/content/drive/MyDrive/workspace/cur-solubility/models/ckpt-2'

In [9]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.10260247942367831}
test set score: {'pearson_r2_score': 0.030558816164417985}


In [10]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

In [11]:
model = create_model()
model.load_weights(latest)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               401920    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Get Docker!

In [4]:
%%capture
!apt-get -qq install docker.io

#Upload Image from deepchem

In [14]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done

echo $IMAGE_URI
echo $WORKSPACE_HOME
echo $PROJECT_ID

gcloud auth activate-service-account --key-file=$WORKSPACE_HOME/creds/kallogjeri-project-345114@appspot.gserviceaccount.com.json
gcloud auth configure-docker --quiet

docker pull deepchemio/deepchem:latest
docker tag deepchemio/deepchem:latest $IMAGE_URI
docker push $IMAGE_URI

# gcloud beta ai custom-jobs local-run \
#   --executor-image-uri=$IMAGE_URI   \
#   --local-package-path=$WORKSPACE_HOME/cur-sol/train \
#   --script=trainer.py \
#   --output-image-uri=gcr.io/$PROJECT_ID/deepchem



kill $(jobs -p)


+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[0000] containerd config version `1` has been deprecated and will be removed in containerd v2.0, please switch to version `2`, see https://github.com/containerd/containerd/blob/main/docs/PLUGINS.md#version-header 
WARN[2023-08-15T22:40:21.159996769Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2023-08-15T22:40:21.160314106Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2023-08-15T22:40:21.211089046Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2023-08-15T22:40:21.211561394Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
ERRO[2023-08-15T22:40:21.215455719Z] AUFS was not found in /proc/filesystems       storage-driver=aufs
ERRO[2023-08-15T22:40:21.228018297Z] failed to mount overlay: invalid a

In [7]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done

gcloud auth activate-service-account --key-file=$WORKSPACE_HOME/creds/kallogjeri-project-345114@appspot.gserviceaccount.com.json #<--Change this
gcloud auth configure-docker --quiet

gcloud ai custom-jobs local-run \
  --executor-image-uri=$IMAGE_URI   \
  --local-package-path=$WORKSPACE_HOME/cur-sol/train \
  --script=trainer.py \
  --output-image-uri=gcr.io/$PROJECT_ID/deepchem



kill $(jobs -p)



+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[0000] containerd config version `1` has been deprecated and will be removed in containerd v2.0, please switch to version `2`, see https://github.com/containerd/containerd/blob/main/docs/PLUGINS.md#version-header 
WARN[2023-08-15T23:28:28.518502316Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2023-08-15T23:28:28.518810915Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2023-08-15T23:28:28.553203552Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2023-08-15T23:28:28.553401615Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
ERRO[2023-08-15T23:28:28.556048733Z] AUFS was not found in /proc/filesystems       storage-driver=aufs
ERRO[2023-08-15T23:28:28.568915649Z] failed to mount overlay: invalid a

In [11]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID)
job = aiplatform.CustomContainerTrainingJob(
    staging_bucket="gs://curated_sol/model",
    display_name="custom-job",
    container_uri=IMAGE_URI
)

job.run(
  replica_count=1,
  machine_type='n1-standard-4',
  accelerator_type='NVIDIA_TESLA_V100',
  accelerator_count=1)


# job.run(
#   replica_count=1,
#   machine_type='n1-standard-4',
#   accelerator_type='NVIDIA_TESLA_V100',
#   accelerator_count=4)


# job.run(
#   replica_count=3,
#   machine_type='n1-standard-16',
#   accelerator_type='NVIDIA_TESLA_V100',
#   accelerator_count=2)




Training Output directory:
gs://curated_sol/model/aiplatform-custom-training-2023-08-16-03:29:36.455 


INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://curated_sol/model/aiplatform-custom-training-2023-08-16-03:29:36.455 
ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x78229ac38c10>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 111, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 87, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 234, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 182, in get
    raise exceptions.TransportError(
google.auth.exceptions.Trans

ServiceUnavailable: ignored

In [ ]:
  !gcloud ai models upload \
  --region=$LOCATION \
  --display-name=$MODEL_NAME \
  --container-image-uri=$IMAGE_URI
  #--artifact-uri=gs://curated_sol/model